In [ ]:
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from functools import partial
from astroquery.jplhorizons import Horizons
from mpl_toolkits.mplot3d import Axes3D

import numpy as np
from numpy import apply_along_axis as thread
from numpy.linalg import norm

np.set_printoptions(sign=' ', linewidth=100, precision=4, suppress=True)
plt.style.use('dark_background')

In [8]:
%run Body.py
%run constants.py
%run rescale_g.py
%run four_body_system.py

In [9]:
mass_unit = sun_mass_kg
length_unit = AU_m
time_unit = day_s

duration = 2.0*365.0
fps = 120.0
t_eval = np.linspace(0.0, duration, num=int(fps*duration))

G = rescale_g(mass_unit, length_unit, time_unit)
print(f'Gravitational Constant In Our Units: {G}')

Gravitational Constant In Our Units: 0.00029600143285188574


In [10]:
moon_info  = Horizons(id= moon_id, id_type= moon_id_type)
earth_info = Horizons(id=earth_id, id_type=earth_id_type)
mars_info  = Horizons(id= mars_id, id_type= mars_id_type)
sun_info   = Horizons(id=  sun_id, id_type=  sun_id_type)

In [11]:
moon  = Body( moon_mass_kg,  moon_info, mass_scale=mass_unit)
earth = Body(earth_mass_kg, earth_info, mass_scale=mass_unit)
mars  = Body( mars_mass_kg,  mars_info, mass_scale=mass_unit)
sun   = Body(  sun_mass_kg,   sun_info, mass_scale=mass_unit)

In [12]:
mu = sun.mass, earth.mass, moon.mass, mars.mass
df = partial(four_body_system, mu=mu, G=G)

In [18]:
state0 = np.hstack((
    sun.pos, earth.pos, moon.pos, mars.pos,
    sun.vel, earth.vel, moon.vel, mars.vel
))

sol = solve_ivp(df, [0.0, duration], state0, rtol=1.0e-9, atol=1.0e-9)#, t_eval=t_eval, rtol=1.0e-9, atol=1.0e-9)
if not sol.success:
    print(sol.message)

x0, x1, x2, x3, v0, v1, v2, v3 = np.vsplit(sol.y, 8)
t = sol.t

p0_T = 0.5 *   sun.mass * thread(norm, 0, v0) ** 2
p1_T = 0.5 * earth.mass * thread(norm, 0, v1) ** 2
p2_T = 0.5 *  mars.mass * thread(norm, 0, v2) ** 2
p3_T = 0.5 *  moon.mass * thread(norm, 0, v3) ** 2

p0_V = -G*(mu[1]/thread(norm,0,x0-x1)+mu[2]/thread(norm,0,x0-x2)+mu[3]/thread(norm,0,x0-x3))
p1_V = -G*(mu[0]/thread(norm,0,x1-x0)+mu[2]/thread(norm,0,x1-x2)+mu[3]/thread(norm,0,x1-x3))
p2_V = -G*(mu[0]/thread(norm,0,x2-x0)+mu[1]/thread(norm,0,x2-x1)+mu[3]/thread(norm,0,x2-x3))
p3_V = -G*(mu[0]/thread(norm,0,x3-x0)+mu[1]/thread(norm,0,x3-x1)+mu[2]/thread(norm,0,x3-x2))

E = p0_T+p0_V + p1_T+p1_V + p2_T+p2_V + p3_T+p3_V

Required step size is less than spacing between numbers.


In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(16,16))
ax = fig.add_subplot(111, projection='3d')

ax.plot(x1[0,:]-x0[0,:], x1[1,:]-x0[1,:], x1[2,:]-x0[2,:], label=r'$P_1$')
ax.plot(x2[0,:]-x0[0,:], x2[1,:]-x0[1,:], x2[2,:]-x0[2,:], label=r'$P_2$')
ax.plot(x3[0,:]-x0[0,:], x3[1,:]-x0[1,:], x3[2,:]-x0[2,:], label=r'$P_3$')


# ax.plot(p0_pos[0,:], p0_pos[1,:], p0_pos[2,:], label=r'$P_0$')
# ax.plot(p1_pos[0,:], p1_pos[1,:], p1_pos[2,:], label=r'$P_1$')

plt.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(32,32), constrained_layout=True)

axs[0,0].set_title(r'Sun')
axs[0,0].plot(t, x0[0,:], label='x')
axs[0,0].plot(t, x0[1,:], label='y')
axs[0,0].plot(t, x0[2,:], label='z')
axs[0,0].legend()
axs[0,0].set_xlabel('t [d]')
axs[0,0].set_ylabel("'x' [AU]")
axs[0,0].grid()

axs[0,0].set_title(r'Earth')
axs[0,0].plot(t, x1[0,:], label='x')
axs[0,0].plot(t, x1[1,:], label='y')
axs[0,0].plot(t, x1[2,:], label='z')
axs[0,0].legend()
axs[0,0].set_xlabel('t [d]')
axs[0,0].set_ylabel("'x' [AU]")
axs[0,0].grid()

axs[0,0].set_title(r'Moon')
axs[0,0].plot(t, x2[0,:], label='x')
axs[0,0].plot(t, x2[1,:], label='y')
axs[0,0].plot(t, x2[2,:], label='z')
axs[0,0].legend()
axs[0,0].set_xlabel('t [d]')
axs[0,0].set_ylabel("'x' [AU]")
axs[0,0].grid()

axs[0,0].set_title(r'Mars')
axs[0,0].plot(t, x3[0,:], label='x')
axs[0,0].plot(t, x3[1,:], label='y')
axs[0,0].plot(t, x3[2,:], label='z')
axs[0,0].legend()
axs[0,0].set_xlabel('t [d]')
axs[0,0].set_ylabel("'x' [AU]")
axs[0,0].grid()

plt.show()